In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
import keras
# from https://github.com/RohitSaha/VGG_Imagenet_Weights_GrayScale_Images/blob/master/convert_vgg_grayscale.py
from keras.applications.vgg16 import VGG16
from keras.models import load_model, Model
from keras.layers import Input, Conv2D, MaxPooling2D

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import sys
from PIL import Image, ImageEnhance
from easydict import EasyDict

from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
# # DO NOT CHANGE-------------------------------------------------------------------------------------------

# # creates new weights for a 1 channel image
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
# model = VGG16(weights='imagenet', include_top=False)

# # Block1_conv1 weights are of the format [3, 3, 3, 64] -> this is for RGB images
# # For grayscale, format should be [3, 3, 1, 64]. Weighted average of the features has to be calculated across channels.
# # RGB weights: Red 0.2989, Green 0.5870, Blue 0.1140

# # getting weights of block1 conv1.
# block1_conv1 = model.get_layer('block1_conv1').get_weights()
# weights, biases = block1_conv1

# # :weights shape = [3, 3, 3, 64] - (0, 1, 2, 3)
# # convert :weights shape to = [64, 3, 3, 3] - (3, 2, 0, 1)
# weights = np.transpose(weights, (3, 2, 0, 1))


# kernel_out_channels, kernel_in_channels, kernel_rows, kernel_columns = weights.shape

# # Dimensions : [kernel_out_channels, 1 (since grayscale), kernel_rows, kernel_columns]
# grayscale_weights = np.zeros((kernel_out_channels, 1, kernel_rows, kernel_columns))

# # iterate out_channels number of times
# for i in range(kernel_out_channels):

# 	# get kernel for every out_channel
# 	get_kernel = weights[i, :, :, :]

# 	temp_kernel = np.zeros((3, 3))

# 	# :get_kernel shape = [3, 3, 3]
# 	# axis, dims = (0, in_channel), (1, row), (2, col)

# 	# calculate weighted average across channel axis
# 	in_channels, in_rows, in_columns = get_kernel.shape

# 	for in_row in range(in_rows):
# 		for in_col in range(in_columns):
# 			feature_red = get_kernel[0, in_row, in_col]
# 			feature_green = get_kernel[1, in_row, in_col]
# 			feature_blue = get_kernel[2, in_row, in_col]

# 			# weighted average for RGB filter
# 			total = (feature_red * 0.2989) + (feature_green * 0.5870) + (feature_blue * 0.1140)

# 			temp_kernel[in_row, in_col] = total


# 	# :temp_kernel is a 3x3 matrix [rows x columns]
# 	# add an axis at the end to specify in_channel as 1

# 	# 2 ways of doing this,

# 	# First: Add axis directly at the end of :temp_kernel to make its shape: [3, 3, 1], but this might be 
# 	# an issue when concatenating all feature maps

# 	# Second: Add axis at the start of :temp_kernel to make its shape: [1, 3, 3] which is [in_channel, rows, columns]
# 	temp_kernel = np.expand_dims(temp_kernel, axis=0)

# 	# Now, :temp_kernel shape is [1, 3, 3]

# 	# Concat :temp_kernel to :grayscale_weights along axis=0
# 	grayscale_weights[i, :, :, :] = temp_kernel

# # Dimension of :grayscale_weights is [64, 1, 3, 3]
# # In order to bring it to tensorflow or keras weight format, transpose :grayscale_weights

# # dimension, axis of :grayscale_weights = (out_channels: 0), (in_channels: 1), (rows: 2), (columns: 3)
# # tf format of weights = (rows: 0), (columns: 1), (in_channels: 2), (out_channels: 3)

# # Go from (0, 1, 2, 3) to (2, 3, 1, 0)
# grayscale_weights = np.transpose(grayscale_weights, (2, 3, 1, 0)) # (3, 3, 1, 64)

# # combine :grayscale_weights and :biases
# new_block1_conv1 = [grayscale_weights, biases]


# # Reconstruct the layers of VGG16 but replace block1_conv1 weights with :grayscale_weights

# # get weights of all the layers starting from 'block1_conv2'
# vgg16_weights = {}
# for layer in model.layers[2:]:
# 	if "conv" in layer.name:
# 		vgg16_weights["200_" + layer.name] = model.get_layer(layer.name).get_weights()

# del model


# # Custom build VGG16
# input = Input(shape=(200, 200, 1), name='200_input')
# # Block 1
# x = Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(200, 200, 1), data_format="channels_last", name='200_block1_conv1')(input)
# x = Conv2D(64, (3, 3), activation='relu', padding='same', name='200_block1_conv2')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block1_pool')(x)

# # Block 2
# x = Conv2D(128, (3, 3), activation='relu', padding='same', name='200_block2_conv1')(x)
# x = Conv2D(128, (3, 3), activation='relu', padding='same', name='200_block2_conv2')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block2_pool')(x)

# # Block 3
# x = Conv2D(256, (3, 3), activation='relu', padding='same', name='200_block3_conv1')(x)
# x = Conv2D(256, (3, 3), activation='relu', padding='same', name='200_block3_conv2')(x)
# x = Conv2D(256, (3, 3), activation='relu', padding='same', name='200_block3_conv3')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block3_pool')(x)

# # Block 4
# x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block4_conv1')(x)
# x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block4_conv2')(x)
# x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block4_conv3')(x)
# x = MaxPooling2D((2, 2), strides=(2, 2), name='200_block4_pool')(x)

# # Block 5
# x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block5_conv1')(x)
# x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block5_conv2')(x)
# x = Conv2D(512, (3, 3), activation='relu', padding='same', name='200_block5_conv3')(x)
# x = MaxPooling2D((8, 8), strides=(8, 8), name='200_block5_pool')(x)

# base_model = Model(inputs=input, outputs=x)

# base_model.get_layer('200_block1_conv1').set_weights(new_block1_conv1)
# for layer in base_model.layers[2:]:
# 	if 'conv' in layer.name:
# 		base_model.get_layer(layer.name).set_weights(vgg16_weights[layer.name])

# #base_model.summary()

# #print base_model.get_layer('block3_conv2').get_weights()
# base_model.save('vgg_grayscale_200.hdf5')

# Change {args.dataset, np.array colors}

In [3]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", type=str, default = 'Batches/testing',
    help="path to input dataset")
ap.add_argument("-e", "--epochs", type=int, default=25,
    help="# of epochs to train our network for")
ap.add_argument("-p", "--plot", type=str, default="plot.png",
    help="path to output loss/accuracy plot")
args = args = EasyDict({
    "dataset": 'Batches/Test_Batch_1',
    "epochs": 25,
    'plot':'plot2_2582.png'
})

In [16]:
print ('Total number of images for train, valid, and test:', len([name for name in os.listdir('Batches/Test_Batch_1/Prolif')]) + \
        len([name for name in os.listdir('Batches/Test_Batch_1/Senescent')]))


Total number of images for train, valid, and test: 5164


In [17]:
cv2. __version__

'4.2.0'

In [6]:
print("Loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:

    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    
    image = cv2.imread(imagePath)
    data.append(image)
    labels.append(label)
    f1_image = cv2.flip(image,0)    
    labels.append(label)
    data.append(f1_image)
    
    r1_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    labels.append(label)
    data.append(r1_image)
    f2_image = cv2.flip(r1_image,0)
    labels.append(label)
    data.append(f2_image)
    
    r2_image = cv2.rotate(r1_image, cv2.ROTATE_90_CLOCKWISE)
    labels.append(label)
    data.append(r2_image)
    f3_image = cv2.flip(r2_image,0)
    labels.append(label)
    data.append(f3_image)
    
    r3_image = cv2.rotate(r2_image, cv2.ROTATE_90_CLOCKWISE)
    labels.append(label)
    data.append(r3_image)
    f4_image = cv2.flip(r3_image,0)
    labels.append(label)
    data.append(f4_image)


    
    
# ------------------------------hto garbage---------------
#     load the image, swap color channels, and resize it to be a fixed
#     128x128 pixels while ignoring aspect ratio
#     competing = Image.open(imagePath)
#     compete = np.array(competing)
#     compete = np.expand_dims(compete, axis = 2)
#     print('type', type(imagePath))
#     print(type(image))
#     #image = Image.open(imagePath)
#     print('image shape: ', np.array(image).shape)
#     print('images dimesnions: ',image.ndim)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(image, (200, 200))
#     print('image dimensions: ', np.array(image).shape)
# ---------------------------------------------------------------


    # update the data and labels lists, respectively
#-------------------------------------------------------------------------------------------------------------------------------
#     img = Image.open(imagePath)
#     image = np.array(img)

#------------------------------------------------------------------------------------------------------------------------------
 



Loading images...


In [7]:
print(len(labels), 'images points')

41312 images points


In [9]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
onehot_encoded
#len(onehot_encoded)

# convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labelss = lb.fit_transform(labels)
#labelss = np_utils.to_categorical(labelss)

(trainX, testX, trainY, testY) = train_test_split(data, onehot_encoded,test_size=0.1, random_state=42)
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY, test_size = 0.1, random_state = 42)


C:\Users\bryan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
# initialize the training data augmentation object

#-------------------------------------------------------------------------------------------------------------------------------
trainAug = ImageDataGenerator()
#-------------------------------------------------------------------------------------------------------------------------------

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()
# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects

#-------------------------------------------------------------------------------------------------------------------------------
# mean = np.array([123.68, 116.779, 103.939], dtype="float32")
# trainAug.mean = mean
# valAug.mean = mean
#-------------------------------------------------------------------------------------------------------------------------------


In [12]:
# load VGG16, ensuring the head FC layer sets are left off, while at
# the same time adjusting the size of the input image tensor to the
# network

#-------------------------------------------------------------------------------------------------------------------------------
# baseModel = VGG16(weights="imagenet", include_top=False,
# 	input_tensor=Input(shape=(200, 200, 1)))
#-------------------------------------------------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------------------------------------------------
baseModel = keras.applications.vgg16.VGG16(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(200, 200, 3),
        pooling=None,
        classes=2)
#-------------------------------------------------------------------------------------------------------------------------------

# show a summary of the base model
#print("[INFO] summary for base model...")
#print(baseModel.summary())

In [13]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(200, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [15]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")

H = model.fit_generator(
	trainAug.flow(trainX, trainY, batch_size=100),
	#steps_per_epoch=30,
	validation_data=valAug.flow(testX, testY),
	validation_steps=10,
	epochs=args["epochs"])

[INFO] compiling model...
[INFO] training head...


MemoryError: 

In [ ]:
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))
# plot the training loss and accuracy
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

In [ ]:
y_pred_class = model.predict(valX)
from sklearn import metrics
print(metrics.accuracy_score(valY, y_pred_class))
print(metrics.confusion_matrix(valY, y_pred_class))